In [1]:
# Load all necessary packages
import sys
sys.path.insert(1, "../")  

import numpy as np
np.random.seed(0)

import time

# Add a sleep statement to wait for imports to be fully loaded
time.sleep(20) 

# Create a TensorFlow session
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
sess = tf.Session()

from aif360.datasets import GermanDataset
time.sleep(20)
from aif360.metrics import BinaryLabelDatasetMetric
time.sleep(20)
from aif360.algorithms.preprocessing import Reweighing
time.sleep(20)

from IPython.display import Markdown, display
time.sleep(20)

# Function to initialize the Reweighing algorithm
def initialize_reweighing(unprivileged_groups, privileged_groups):
    return Reweighing(unprivileged_groups=unprivileged_groups,
                      privileged_groups=privileged_groups)

# Function to initialize the AdversarialDebiasing algorithm
def initialize_adversarial_debiasing(unprivileged_groups, privileged_groups, adversary_loss_weight, num_epochs, batch_size, classifier_num_hidden_units, debias, sess):
    scope_name = "my_scope"
    return AdversarialDebiasing(unprivileged_groups=unprivileged_groups,
                                privileged_groups=privileged_groups,
                                scope_name=scope_name,
                                sess=sess,
                                seed=None,
                                adversary_loss_weight=adversary_loss_weight,
                                num_epochs=num_epochs,
                                batch_size=batch_size,
                                classifier_num_hidden_units=classifier_num_hidden_units,
                                debias=debias)

# Function to initialize the OptimPreproc algorithm
def initialize_optim_preproc(unprivileged_groups, privileged_groups, optimizer, optim_options, verbose):
    return OptimPreproc(optimizer=optimizer,
                        optim_options=optim_options,
                        unprivileged_groups=unprivileged_groups,
                        privileged_groups=privileged_groups,
                        verbose=verbose,
                        seed=None)

# Load dataset
dataset_orig = GermanDataset(
    protected_attribute_names=['age'],
    privileged_classes=[lambda x: x >= 25],
    features_to_drop=['personal_status', 'sex']
)

# Split dataset
dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

# Define privileged and unprivileged groups
privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

# Define metric
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                           unprivileged_groups=unprivileged_groups,
                           privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

# Initialize ALG as None initially
ALG = None

# User-inputted algorithm
algorithm_choice = "Reweighing"

# Initialize algorithm based on user input 
if algorithm_choice == "Reweighing":
    ALG = initialize_reweighing(unprivileged_groups, privileged_groups)
elif algorithm_choice == "AdversarialDebiasing":
    ALG = initialize_adversarial_debiasing(unprivileged_groups, privileged_groups, 
                                           adversary_loss_weight=0.1, num_epochs=50, batch_size=128, 
                                           classifier_num_hidden_units=200, debias=True, sess=sess)
elif algorithm_choice == "OptimPreproc":
    ALG = initialize_optim_preproc(unprivileged_groups, privileged_groups, 
                                   optimizer='my_optimizer', optim_options={'option1': 'value1', 'option2': 'value2'}, 
                                   verbose=False)

# Fit and transform dataset
dataset_transf_train = ALG.fit_transform(dataset_orig_train)

# Compute metric on transformed dataset
metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)

# Display results
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())


2024-04-01 09:08:36.236889: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2024-04-01 09:08:37.120837: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Instructions for updating:
non-resource variables are not supported in the long term


`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[::2, :], raw_df

/opt/hostedtoolcache/Python/3.11.8/x64/lib/python3.11/site-packages/torch/_functorch/deprecated.py:61: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.vmap is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.vmap instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('vmap', 'torch.vmap')


#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.169905


#### Transformed training dataset

Difference in mean outcomes between unprivileged and privileged groups = 0.000000
